<a href="https://colab.research.google.com/github/dapopov-st/RagOverArXiv/blob/main/RAG_over_ArXiv_PDFs_Part6_Mistral_7b_Instruct_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
- Experiment with Mistral 7B Instruct Finetuning.  
- Be careful if finetune on abstracts only as that may not be representative unless initial data RAGs over abstract as well.  Something to test, though.
- The idea is to bootstrap the fine tuning of the model on abstracts by having it predict the title since no labels are available.
Update: Looks like I found some settings/parameters that allow the model to fine tune reasonably well. The ouput produced is sensible, but need to train on more data for longer.

## Get the data and build a Retriever

In [1]:
!pip install -U -q "langchain" "transformers==4.35.0" "datasets==2.12.0" "tokenizers==0.14.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.41.1" "trl==0.4.7" "safetensors>=0.3.1" "arxiv==1.4"
#0.40.2
 #contains highly optimized components some of which are not yet available in PyTorch
!pip install xformers
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 120.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import transformers, datasets, tokenizers, torch
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
transformers.__version__, datasets.__version__, tokenizers.__version__, torch.__version__

('4.35.0', '2.12.0', '0.14.0', '2.1.0+cu118')

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive/')

output_dir = '/content/drive/MyDrive/PdfRag/finetune_output_dir'
logging_dir = '/content/drive/MyDrive/PdfRag/finetune_logging_dir'
index_dir = '/content/drive/MyDrive/PdfRag/finetune_index_dir'

%cd /content/drive/MyDrive/PdfRag
!ls .


Mounted at /content/drive/
/content/drive/MyDrive/PdfRag
 cache										 outputs
 clusterofstars									 rag_index_dir
'Efficient Parallelization Layouts for Large-Scale Distributed Model Training'	 testabstract
 finetune_output_dir								 wandb
 instructionmining


In [4]:
# fastai function to clean GPU memory
import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''


def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()



In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Configure Model, Data, Tokenizer, Trainer

In [7]:
clean_mem()

In [8]:
# BitsAndBytes for 4-bit quantization with NF4-type configuration to load  model in 4-bit precision.
# Will help load the model faster and reduce the memory footprint so that it can be run on Google Colab.
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
#model_id = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(model_id)


model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config, #using bnb_config
    device_map='auto'
)
model.gradient_checkpointing_enable()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fine-tune the model using a small number of parameters of an adopter layer, making the entire process faster and more memory-efficient.

In [9]:
#orig_model=model
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [18]:
# Load the corresponding tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    return_overflowing_tokens=True #adding for shape mismatches
)
tokenizer.padding_side = 'right' #Warning stated to use this with decoder-only; however 'left' made trained model worse! Try right again after prompt adjustment
# tokenizer.pad_token = tokenizer.eos_token
#tokenizer.add_eos_token = True
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_bos_token, tokenizer.add_eos_token
#Adding tokens was messing up Mistral generation!!!

In [11]:
#First LOAD SMALL SLICE FOR TESTING
#Most useful way of slicing suggested here: https://huggingface.co/docs/datasets/v1.11.0/splits.html
import datasets
dataset = datasets.load_dataset('CShorten/ML-ArXiv-Papers',split='train[:1%]')
#dataset = dataset.rename_column('abstract','text')
print(dataset.column_names)
# dataset = dataset['train'].train_test_split(train_size=0.95, seed=42)
# # Rename the default "test" split to "validation"
# dataset['val'] = dataset.pop("test")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.
['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract']


Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
    num_rows: 1176
})

In [12]:

from copy import deepcopy
from random import randrange
from functools import partial

def format_prompt(sample):
    """Given a sample dictionary with keys "title" and "abstract" format into a prompt.

    Args:
      sample: A sample dictionary from a Hugging Face dataset.

    Returns:
      sample: sample dictionary with "text" key for the formatted prompt.
    """
    sample['text']=f"<s>[INST] Write a title for the following abstract {sample['abstract']} [/INST] {sample['title']}</s>"
    return sample



# https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: transformers.AutoTokenizer, max_length: int, dataset: str, seed: int = 42):
    # Format each prompt.
    print("Preprocessing dataset...")
    dataset = dataset.map(format_prompt)


    # https://blog.ovhcloud.com/fine-tuning-llama-2-models-using-a-single-gpu-qlora-and-ai-notebooks/
    def preprocess_batch(batch, tokenizer, max_length):
        return tokenizer(
            batch["text"],
            max_length=max_length,
            truncation=True,
        )


    # Apply preprocessing to each batch of the dataset & and remove "conversations" and "text" fields.
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['Unnamed: 0.1', 'Unnamed: 0','title','abstract']#,'text']
    )


    # Filter out samples that have input_ids exceeding max_length.
    # Not needed as the tokenizer truncates all prompts over max length.
    # dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)


    # Shuffle dataset.
    dataset = dataset.shuffle(seed=seed)


    return dataset





In [13]:
## Testing original model's output
# abstract = "We introduce GAIA, a benchmark for General AI Assistants that, if solved, would represent a milestone in AI research. GAIA proposes real-world questions that require a set of fundamental abilities such as reasoning, multi-modality handling, web browsing, and generally tool-use proficiency. GAIA questions are conceptually simple for humans yet challenging for most advanced AIs: we show that human respondents obtain 92\% vs. 15\% for GPT-4 equipped with plugins. This notable performance disparity contrasts with the recent trend of LLMs outperforming humans on tasks requiring professional skills in e.g. law or chemistry. GAIA's philosophy departs from the current trend in AI benchmarks suggesting to target tasks that are ever more difficult for humans. We posit that the advent of Artificial General Intelligence (AGI) hinges on a system's capability to exhibit similar robustness as the average human does on such questions. Using GAIA's methodology, we devise 466 questions and their answer. We release our questions while retaining answers to 300 of them to power a leader-board available at https://huggingface.co/gaia-benchmark."

# eval_prompt = f'<s>[INST] Write a title for the following abstract {abstract} [/INST]'
# model_input = tokenizer(eval_prompt, return_tensors="pt").to('cuda')

# # For some reason, could not see the ouput when using torch.no_grad()
# tokenizer.decode(orig_model.generate(**model_input, max_new_tokens=2000, temperature=0.8)[0])

# """
# <s><s> [INST] Write a title for the following abstract We introduce GAIA, a benchmark for General AI Assistants that, if solved,
# would represent a milestone in AI research. GAIA proposes real-world questions that require a set of fundamental abilities such
# as reasoning, multi-modality handling, web browsing, and generally tool-use proficiency. GAIA questions are conceptually simple
# for humans yet challenging for most advanced AIs: we show that human respondents obtain 92\% vs. 15\% for GPT-4 equipped with plugins.
# This notable performance disparity contrasts with the recent trend of LLMs outperforming humans on tasks requiring professional
# skills in e.g. law or chemistry. GAIA's philosophy departs from the current trend in AI benchmarks suggesting to target tasks
# that are ever more difficult for humans. We posit that the advent of Artificial General Intelligence (AGI) hinges on a system's
# capability to exhibit similar robustness as the average human does on such questions. Using GAIA's methodology, we devise 466
# questions and their answer. We release our questions while retaining answers to 300 of them to power a leader-board available
#  at https://huggingface.co/gaia-benchmark. [/INST] "Introducing GAIA: A Benchmark for General AI Assistants and the Quest for Artificial General Intelligence"</s>
# """

In [14]:
MAX_LEN = 2048 #tried 2048, maybe a bit too big for GPU
formatted_dataset = deepcopy(dataset).map(format_prompt)
dataset = preprocess_dataset(tokenizer, max_length=MAX_LEN, dataset=dataset)


Map:   0%|          | 0/1176 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/1176 [00:00<?, ? examples/s]

In [15]:
dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1176
})

In [19]:
# gradient_accumulation_steps=4 with learning_rate=2e-4 kind of worked; 16 did not!
#2e-5 +16 diverges
#2e-5 +8 diverges
import transformers, trl
from datetime import datetime

project = "mistral-7b-instruct-finetune"
base_model_name = model_id
run_name = base_model_name + "-" + project
# config = {
#         'warmup_steps':5,
#         'per_device_train_batch_size':1, # Best practice: https://huggingface.co/docs/transformers/main/main_classes/quantization#tips-and-best-practices
#         'gradient_accumulation_steps':16, #may try 1 or 4 (was working)
#         'max_steps':20,
#         'learning_rate':2.5e-5, # Want about 10x smaller than the Mistral learning rate
#         'logging_steps':1, #was 10
#         'bf16':False,
#         'fp16':False,
#         'weight_decay':0.001,
#         'group_by_length':True,
#         'lr_scheduler_type':"cosine"}
# config = {
#         'warmup_steps':5,
#         'per_device_train_batch_size':1, # Best practice: https://huggingface.co/docs/transformers/main/main_classes/quantization#tips-and-best-practices
#         'gradient_accumulation_steps':8, #may try 1 or 4 (was working)
#         'max_steps':20,
#         'learning_rate':2e-4,#2.5e-5, # Want about 10x smaller than the Mistral learning rate
#         'fp16':True,
#         'lr_scheduler_type':"cosine"}
config = {
        'warmup_ratio':0.1,
        'per_device_train_batch_size':2, # Best practice: https://huggingface.co/docs/transformers/main/main_classes/quantization#tips-and-best-practices
        'gradient_accumulation_steps':8, #may try 1 or 4 (was working)
        'max_steps':20,
        'learning_rate':5e-5,#2.5e-5, # Want about 10x smaller than the Mistral learning rate
        'fp16':True,
        'lr_scheduler_type':"linear"}
training_args = transformers.TrainingArguments(
        **config,
        output_dir=output_dir,
        optim="paged_adamw_8bit", #may try paged_adamw_32bit
        #optim="paged_adamw_32bit", #may try paged_adamw_32bit
        logging_dir=logging_dir,        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        logging_steps=1, #was 10
        save_steps=5,                # Save checkpoints every 50 steps
        #evaluation_strategy="steps", # Evaluate the model every logging step
        #eval_steps=2,               # Evaluate and save checkpoints every 50 steps
        do_eval=False,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    )
# training_args = transformers.TrainingArguments(
#     output_dir="outputs",
#     per_device_train_batch_size=1,  # Best practice: https://huggingface.co/docs/transformers/main/main_classes/quantization#tips-and-best-practices
#     gradient_accumulation_steps=8,  # Powers of 2.
#     learning_rate=2e-4,
#     max_grad_norm=1.0,
#     max_steps=20,#40?
#     lr_scheduler_type="cosine",
#     warmup_steps=5,
#     fp16=True,
#     logging_strategy="steps",
#     logging_steps=1,
#     save_strategy="steps",
#     save_steps=10,
#     optim="paged_adamw_8bit",
#     report_to="wandb"
# )


trainer = trl.SFTTrainer(
    model=model,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=dataset,
    #packing  = True
    dataset_text_field = 'text',
    #formatting_func=format_prompt #don't think I need this if using text field
)

results = trainer.train()


# Initialize a run
run = wandb.init(project="mistral7b_arxiv_finetune",
                 job_type="Finetuning",
                 save_code=True,
                 config=config)



/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.852300
2,2.827000
3,2.858400
4,2.897900
5,2.674800
6,2.744700
7,2.800900
8,2.638500
9,2.582700
10,2.538500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

train/epoch,▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/learning_rate,▅██▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
train/loss,▇▇▇█▅▆▇▅▄▃▅▃▂▃▄▄▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,0.27
train/global_step,20


In [20]:
os.listdir(output_dir)

['README.md',
 'adapter_model.bin',
 'adapter_config.json',
 'checkpoint-2',
 'checkpoint-4',
 'checkpoint-6',
 'checkpoint-8',
 'checkpoint-10',
 'checkpoint-5',
 'checkpoint-15',
 'checkpoint-20']

In [22]:
torch.save(model.state_dict(), output_dir+'/'+'final_out.pt')

In [26]:
clean_mem()

In [30]:
#Likely need to restart kernel after training since o.w. run out of CUDA memory
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = model_id
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

#ft_model = PeftModel.from_pretrained(base_model, output_dir+'/'+'checkpoint-10')
ft_model = PeftModel.from_pretrained(base_model, output_dir+'/'+'checkpoint-20')


In [23]:
abstract = "We introduce GAIA, a benchmark for General AI Assistants that, if solved, would represent a milestone in AI research. GAIA proposes real-world questions that require a set of fundamental abilities such as reasoning, multi-modality handling, web browsing, and generally tool-use proficiency. GAIA questions are conceptually simple for humans yet challenging for most advanced AIs: we show that human respondents obtain 92\% vs. 15\% for GPT-4 equipped with plugins. This notable performance disparity contrasts with the recent trend of LLMs outperforming humans on tasks requiring professional skills in e.g. law or chemistry. GAIA's philosophy departs from the current trend in AI benchmarks suggesting to target tasks that are ever more difficult for humans. We posit that the advent of Artificial General Intelligence (AGI) hinges on a system's capability to exhibit similar robustness as the average human does on such questions. Using GAIA's methodology, we devise 466 questions and their answer. We release our questions while retaining answers to 300 of them to power a leader-board available at https://huggingface.co/gaia-benchmark."

eval_prompt = f'<s>[INST] Write a title for the following abstract {abstract} [/INST]'
model_input = tokenizer(eval_prompt, return_tensors="pt").to('cuda')

# For some reason, could not see the ouput when using torch.no_grad()
tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=2000, temperature=0.8)[0])
#"GAIA: A Benchmark for General AI Assistants and the Quest for Artificial General Intelligence"

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s><s> [INST] Write a title for the following abstract We introduce GAIA, a benchmark for General AI Assistants that, if solved, would represent a milestone in AI research. GAIA proposes real-world questions that require a set of fundamental abilities such as reasoning, multi-modality handling, web browsing, and generally tool-use proficiency. GAIA questions are conceptually simple for humans yet challenging for most advanced AIs: we show that human respondents obtain 92\\% vs. 15\\% for GPT-4 equipped with plugins. This notable performance disparity contrasts with the recent trend of LLMs outperforming humans on tasks requiring professional skills in e.g. law or chemistry. GAIA\'s philosophy departs from the current trend in AI benchmarks suggesting to target tasks that are ever more difficult for humans. We posit that the advent of Artificial General Intelligence (AGI) hinges on a system\'s capability to exhibit similar robustness as the average human does on such questions. Using G

In [24]:
abstract = 'Multimedia generation approaches occupy a prominent place in artificial intelligence research. Text-to-image models achieved high-quality results over the last few years. However, video synthesis methods recently started to develop. This paper presents a new two-stage latent diffusion text-to-video generation architecture based on the text-to-image diffusion model. The first stage concerns keyframes synthesis to figure the storyline of a video, while the second one is devoted to interpolation frames generation to make movements of the scene and objects smooth. We compare several temporal conditioning approaches for keyframes generation. The results show the advantage of using separate temporal blocks over temporal layers in terms of metrics reflecting video generation quality aspects and human preference. The design of our interpolation model significantly reduces computational costs compared to other masked frame interpolation approaches. Furthermore, we evaluate different configurations of MoVQ-based video decoding scheme to improve consistency and achieve higher PSNR, SSIM, MSE, and LPIPS scores. Finally, we compare our pipeline with existing solutions and achieve top-2 scores overall and top-1 among open-source solutions: CLIPSIM = 0.2976 and FVD = 433.054. Project page: https://ai-forever.github.io/kandinsky-video/'
eval_prompt = f'<s>[INST] Write a title for the following abstract {abstract} [/INST]'
model_input = tokenizer(eval_prompt, return_tensors="pt").to('cuda')

# For some reason, could not see the ouput when using torch.no_grad()
tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=2000, temperature=0.8)[0])
# "Latent Diffusion Text-to-Video Generation with Temporal Conditioning and MoVQ-based Decoding"


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s><s> [INST] Write a title for the following abstract Multimedia generation approaches occupy a prominent place in artificial intelligence research. Text-to-image models achieved high-quality results over the last few years. However, video synthesis methods recently started to develop. This paper presents a new two-stage latent diffusion text-to-video generation architecture based on the text-to-image diffusion model. The first stage concerns keyframes synthesis to figure the storyline of a video, while the second one is devoted to interpolation frames generation to make movements of the scene and objects smooth. We compare several temporal conditioning approaches for keyframes generation. The results show the advantage of using separate temporal blocks over temporal layers in terms of metrics reflecting video generation quality aspects and human preference. The design of our interpolation model significantly reduces computational costs compared to other masked frame interpolation ap